In [1]:
!pip install twilio
!pip install geopy
!pip install scikit-learn
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.8 MB/s eta 0:00:00


In [14]:
import pandas as pd
from twilio.rest import Client
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from IPython.core.display import display, HTML





In [15]:
# Load the dataset
data_path = 'katraj_osm_disaster_preparedness_data_with_features.csv'
df = pd.read_csv(data_path)

In [16]:
# Features for the Random Forest Model
features = ['Rainfall_Intensity', 'Elevation', 'Distance_to_Water_Body']
target = 'Flood_Prone'

In [17]:
# Split the dataset into training and testing sets
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [19]:
# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [20]:
# Make predictions on the test set
y_pred = rf_model.predict(X_test)

In [12]:
# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       121
           1       1.00      0.83      0.91         6

    accuracy                           0.99       127
   macro avg       1.00      0.92      0.95       127
weighted avg       0.99      0.99      0.99       127

Accuracy Score: 0.9921259842519685


In [8]:
# Function to find the nearest facilities by category (hospitals, clinics, grocery)
def get_nearby_locations_by_category(user_lat, user_lon, data, category, required_count):
    locations = []
    for index, row in data.iterrows():
        if row['Type'].lower() == category.lower():
            location_latlon = (row['Latitude'], row['Longitude'])
            user_latlon = (user_lat, user_lon)

            # Calculate the distance between user and location
            distance = geodesic(user_latlon, location_latlon).km
            locations.append({
                'Name': row['Name'],
                'Type': row['Type'],
                'Distance_km': distance,
                'Maps_Link': row['Maps_Link']
            })

    # Sort by distance and return the top locations required
    locations = sorted(locations, key=lambda x: x['Distance_km'])[:required_count]

    return locations



In [10]:
# Function to check flood status and list nearby essential services
def check_flood_and_alert(user_lat, user_lon, data, model):
    # Filter data to find the row that corresponds to the user's location (latitude and longitude)
    user_data = data[(data['Latitude'] == user_lat) & (data['Longitude'] == user_lon)]

    # If no matching location found, return a message
    if user_data.empty:
        return "Location data not found for your area."

    # Extract the relevant features for prediction from the data
    user_input_features = user_data[['Rainfall_Intensity', 'Elevation', 'Distance_to_Water_Body']].iloc[0].to_dict()

    # Create a DataFrame with the same columns used during training
    input_data = pd.DataFrame([user_input_features])

    # Make prediction using the Random Forest model
    is_flood_prone = model.predict(input_data)[0]

    message_body = ""

    if is_flood_prone == 1:
        # If flood-prone, prepare alert message
        message_body += "ALERT: Your area is flood-prone! Please take necessary precautions.\n\n"
    else:
        message_body += "Good News: Your area is not currently flood-prone. In Case of Emergency : \n\n"

    # Get nearby hospitals, clinics/medical stores/dentists, and grocery stores
    nearby_hospitals = get_nearby_locations_by_category(user_lat, user_lon, data, 'hospital', 4)
    nearby_clinics = get_nearby_locations_by_category(user_lat, user_lon, data, 'clinic', 3)
    nearby_medical_stores = get_nearby_locations_by_category(user_lat, user_lon, data, 'medical store', 3)
    nearby_dentists = get_nearby_locations_by_category(user_lat, user_lon, data, 'dentist', 3)
    nearby_groceries = get_nearby_locations_by_category(user_lat, user_lon, data, 'grocery', 3)

    # Display nearby hospitals (ensure we have at least 4)
    message_body += "Hospitals:\n"
    if len(nearby_hospitals) >= 4:
        for hospital in nearby_hospitals:
            message_body += f"{hospital['Type'].capitalize()}: {hospital['Name']} - {hospital['Distance_km']:.2f} km away\n"
            message_body += f"Maps Link: {hospital['Maps_Link']}\n"
            message_body += "-----\n"
    else:
        message_body += "Not enough hospitals found nearby.\n"

    # Display nearby clinics or medical stores/dentists (ensure we have 3)
    message_body += "\nClinics/Medical Stores/Dentists:\n"
    combined_clinics = nearby_clinics + nearby_medical_stores + nearby_dentists
    if len(combined_clinics) >= 3:
        for clinic in combined_clinics[:3]:
            message_body += f"{clinic['Type'].capitalize()}: {clinic['Name']} - {clinic['Distance_km']:.2f} km away\n"
            message_body += f"Maps Link: {clinic['Maps_Link']}\n"
            message_body += "-----\n"
    else:
        message_body += "Not enough clinics, medical stores, or dentists found nearby.\n"

    # Display nearby grocery stores (ensure we have 3)
    message_body += "\nGrocery Stores:\n"
    if len(nearby_groceries) >= 3:
        for grocery in nearby_groceries:
            message_body += f"{grocery['Type'].capitalize()}: {grocery['Name']} - {grocery['Distance_km']:.2f} km away\n"
            message_body += f"Maps Link: {grocery['Maps_Link']}\n"
            message_body += "-----\n"
    else:
        message_body += "Not enough grocery stores found nearby.\n"

    return message_body






In [ ]:
# Function to send WhatsApp messages using Twilio to all numbers in CSV
def send_whatsapp_messages(body, phone_number):
    # Twilio credentials from your Twilio account
    account_sid = 'Enter your own Twilio Account'
    auth_token = 'Enter your authentication token'
    client = Client(account_sid, auth_token)

    # Ensure phone number is in correct format: remove any decimals or extraneous characters
    phone_number = str(phone_number).replace('.0', '').replace('+', '').strip()

    # Send message to the user's number
    message = client.messages.create(
        body=body,
        from_='whatsapp:+14155238886',  # Twilio's WhatsApp sandbox number
        to=f'whatsapp:+{phone_number}'  # Send to the user's phone number in E.164 format
    )
    print(f"Message sent successfully to {phone_number}. SID:", message.sid)


# Function to iterate over users and send personalized alerts
def send_personalized_alerts(df, model):
    # Load phone numbers and user locations from CSV
    numbers_df = pd.read_csv('numbers.csv')

    for index, row in numbers_df.iterrows():
        phone_number = row['phone_number']  # Assuming the column is named 'phone_number'
        user_lat = row['Latitude']  # Assuming the user's latitude is in the 'Latitude' column
        user_lon = row['Longitude']  # Assuming the user's longitude is in the 'Longitude' column

        # Check flood status and alert with nearby facilities
        personalized_message = check_flood_and_alert(user_lat, user_lon, df, model)

        # Send the personalized message to the user's phone
        send_whatsapp_messages(personalized_message, phone_number)



In [13]:
# Send personalized alerts to each user in the CSV
send_personalized_alerts(df, rf_model)

Message sent successfully to 918698920168. SID: SM64b30e47b2ea21fec5c5b29a969560dd
Message sent successfully to 919373288538. SID: SMb18aa7312597677fb76a891a9680ff0f
Message sent successfully to 919403635930. SID: SM4034fce979e6ef703a86f2c910bddd13
Message sent successfully to 918956144030. SID: SM49fa414ec2d61466736206b517025b8b
Message sent successfully to 919767672283. SID: SM0426152d11fdf6a19d0743c89532fb90
